In [11]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../src/utils")
from utils import SimpleGraphVoltDatasetLoader_Lazy
from torch_geometric_temporal.signal import temporal_signal_split
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import A3TGCN
from tqdm import tqdm

In [12]:
trafo_id = "T1330"
epochs = 25
num_timesteps_in = 12
num_timesteps_out = 4
train_ratio = 0.7
test_ratio_vs_eval_ratio = 0.5
learning_rate = 0.01

In [13]:
#get dateime string of now
now = pd.Timestamp.now().strftime("%Y%m%d%H%M%S")

In [14]:
print("Loading data...", end="")
loader = SimpleGraphVoltDatasetLoader_Lazy(trafo_id, num_timesteps_in, num_timesteps_out)
print(" done")
loader_data_index = loader.snapshot_index
# loader_data = loader.get_dataset(num_timesteps_in=num_timesteps_in, num_timesteps_out=num_timesteps_out)

Loading data...Voltage index: 5
 done


In [15]:
train_dataset, test_eval_dataset = loader.temporal_signal_split_lazy(loader_data_index, train_ratio)
test_dataset, eval_dataset = loader.temporal_signal_split_lazy(test_eval_dataset, test_ratio_vs_eval_ratio)

In [21]:
def run_avg_loss(dataset, loader, loss_fn):#, device):
   
    loss_run_avg = 0
    for snapshot_i in dataset:
        snapshot = loader.get_snapshot(snapshot_i)
        #snapshot.to(device) #a treba?
        out = torch.zeros_like(snapshot.y)
        for node_index in range(len(snapshot.x)):
            predicted_voltage = np.average(snapshot.x[node_index, loader.voltage_index, :]) / len(snapshot.x[node_index, loader.voltage_index, :])
            out[node_index, :] = predicted_voltage
            #print(snapshot.x[node_index, loader.voltage_index, :], predicted_voltage)

        loss = loss_fn()(out, snapshot.y)
        loss_run_avg += loss.item()

    return loss_run_avg

print(run_avg_loss(train_dataset, loader, torch.nn.L1Loss))
        

22591.62607413903
